In [1]:
# Install necessary libraries
%pip install os
%pip install pandas
%pip install nltk
%pip install re
%pip install gensim
%pip install pyLDAvis
%pip install pyLDAvis.gensim
%pip install textblob

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


...

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pyLDAvis.gensim (from versions: none)
ERROR: No matching distribution found for pyLDAvis.gensim


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: textblob in c:\users\ethan\anaconda3\lib\site-packages (0.19.0)



In [ ]:
# Import necessary libraries
import os
import pandas as pd
import nltk
import re
import gensim
from gensim import corpora
from gensim.models import LdaModel, CoherenceModel
from textblob import TextBlob

# Download NLTK resources
nltk.download('stopwords')
nltk.download('punkt')

# Define constants
DATA_FOLDER = "processed_data"
DEFAULT_FILE = "steam_reviews.csv"
SENTIMENT_CSV = os.path.join(DATA_FOLDER, "sentiment_results.csv")
LDA_TOPICS_CSV = os.path.join(DATA_FOLDER, "lda_topics.csv")

# Ensure data folder exists
os.makedirs(DATA_FOLDER, exist_ok=True)

# Custom stopword list based on frequency graph
stop_list = nltk.corpus.stopwords.words('english')
stop_list += [
    "game", "play", "like", "good", "best", "one", "great", "really", "get", "time",
    "ever", "played", "playing", "would", "even", "much", "hours", "love", "still",
    "buy", "amazing", "in", "u", "it", "xd", "lol", "ive", "im", "could", "also", "many",
    "im", "lol", "cant", "ing"
]

# Function to preprocess and tokenize reviews
def preprocess_reviews(df, text_column='review_text'):
    """ Tokenizes, removes stopwords, and cleans review text. """
    # Drop missing values in the review_text column
    df = df.dropna(subset=[text_column])
    
    # Create a copy of the dataframe with tokenized reviews
    df_processed = df.copy()
    
    # Tokenize and clean reviews
    df_processed['tokens'] = df[text_column].astype(str).apply(nltk.word_tokenize)
    df_processed['tokens'] = df_processed['tokens'].apply(lambda x: [w.lower() for w in x if re.search('^[a-z]+$', w) and w.lower() not in stop_list])
    
    return df_processed

# Load the default dataset
print(f"Loading default dataset from {DEFAULT_FILE}...")
df = pd.read_csv(DEFAULT_FILE)

# Preprocess the reviews
df_processed = preprocess_reviews(df)

# Perform sentiment analysis
print("Performing sentiment analysis...")
df_processed['sentiment'] = df_processed['review_text'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)
df_processed['sentiment_category'] = df_processed['sentiment'].apply(lambda x: 'Positive' if x > 0 else 'Negative' if x < 0 else 'Neutral')

# Save sentiment results
df_processed.to_csv(SENTIMENT_CSV, index=False)
print(f"Sentiment analysis results saved to {SENTIMENT_CSV}")

# Extract tokenized documents
docs = df_processed['tokens'].tolist()

# Create Dictionary
dictionary = corpora.Dictionary(docs)

# Convert documents into Bag of Words representation
corpus_bow = [dictionary.doc2bow(doc) for doc in docs]

# Train LDA Model
print("Training LDA model...")
num_topics = 10  # Adjust as needed
lda_model = LdaModel(corpus=corpus_bow, id2word=dictionary, num_topics=num_topics, passes=10, random_state=42)

# Print topics
for idx, topic in lda_model.print_topics():
    print(f"Topic {idx}: {topic}")

# Save topic words to CSV
topic_words = []
for idx, topic in lda_model.show_topics(num_topics=10, num_words=10, formatted=False):
    for word, weight in topic:
        topic_words.append({"topic": idx, "word": word, "weight": weight})

pd.DataFrame(topic_words).to_csv(LDA_TOPICS_CSV, index=False)
print(f"LDA topics saved to {LDA_TOPICS_CSV}")

print("Processing complete. CSV files have been saved for frontend visualization.")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ethan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ethan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Loading default dataset from steam_reviews.csv...
